In [1]:
import sys
from pathlib import Path

# Get the absolute path of the current notebook
notebook_path = Path().resolve()

# Get the project root directory (which is the parent of the 'notebooks' directory)
project_root = notebook_path.parent

# Add BOTH the project root and the src directory to the Python path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
if str(project_root / 'src') not in sys.path:
    sys.path.append(str(project_root / 'src'))

# Now, we can import our modules
from src.data_handling import DataHandler
from src.mlp_model import MLPModel

Using MPS device (Apple Silicon GPU)


#### Initialize DataHandler
This cell creates an instance of the `DataHandler` class. It will load the dataset, define the feature/target sets, and pre-fit the necessary data processing objects. The test year is set to 2020.

In [2]:
dh = DataHandler(test_year=2020)

DataHandler initialized - Using 52 features - Test year: 2020


### **Model: MLP with 1 Hidden Layer**

#### Initialize MLPModel Handler
Here, we create an instance of the `MLPModel` class, explicitly setting `depth=2`. The `model_name` ensures all artifacts for this architecture are saved to unique files.

In [3]:
mlp_depth1 = MLPModel(dh, model_name='mlp_depth1', depth=1)

MLPModel initialized with model name mlp_depth1 and depth 1.
Optuna study will be stored in  : /Users/arvindsuresh/Documents/Github/Election-prediction-May-2025/2020-results-20251023/optuna/mlp_depth1_study.pkl
Trained model will be stored in : /Users/arvindsuresh/Documents/Github/Election-prediction-May-2025/2020-results-20251023/models/mlp_depth1_model.pth
Final preds will be stored in   : /Users/arvindsuresh/Documents/Github/Election-prediction-May-2025/2020-results-20251023/preds/mlp_depth1_preds.csv


#### Run Optuna Hyperparameter Study (Depth=2)
This cell runs the Optuna study. For this specific 2-layer architecture, it will find the optimal learning rate, weight decay, batch size, and the number of neurons and dropout rates for both hidden layers.

In [4]:
mlp_depth1.run_optuna_study(
    n_trials=50,
    timeout=30,               # Stop study after 30 minutes
    max_epochs=40,            # Max epochs to train each trial
    min_resource=4,          # Min epochs before pruning
    reduction_factor=2,
    use_pca=False
)

[I 2025-10-23 10:45:58,807] A new study created in memory with name: mlp_depth1_study
[I 2025-10-23 10:46:19,248] Trial 2 pruned. 
[I 2025-10-23 10:46:19,601] Trial 4 pruned. 
[I 2025-10-23 10:46:19,986] Trial 6 pruned. 
[I 2025-10-23 10:46:21,286] Trial 8 pruned. 
[I 2025-10-23 10:46:29,581] Trial 3 pruned. 
[I 2025-10-23 10:46:36,841] Trial 12 pruned. 
[I 2025-10-23 10:46:37,939] Trial 9 pruned. 
[I 2025-10-23 10:46:39,964] Trial 10 pruned. 
[I 2025-10-23 10:46:40,791] Trial 11 pruned. 
[I 2025-10-23 10:46:51,025] Trial 18 pruned. 
[I 2025-10-23 10:46:51,216] Trial 13 pruned. 
[I 2025-10-23 10:46:53,507] Trial 16 pruned. 
[I 2025-10-23 10:46:58,042] Trial 14 pruned. 
[I 2025-10-23 10:47:01,666] Trial 15 pruned. 
[I 2025-10-23 10:47:04,652] Trial 20 pruned. 
[I 2025-10-23 10:47:06,876] Trial 21 pruned. 
[I 2025-10-23 10:47:17,452] Trial 22 pruned. 
[I 2025-10-23 10:47:24,865] Trial 24 pruned. 
[I 2025-10-23 10:47:27,026] Trial 25 pruned. 
[I 2025-10-23 10:47:34,735] Trial 23 pruned. 


--------------------
Study concluded and saved to /Users/arvindsuresh/Documents/Github/Election-prediction-May-2025/2020-results-20251023/optuna/mlp_depth1_study.pkl.
Best trial: 42
Best loss: 1.0016956478357315
Best params: {'learning_rate': 0.01040797308412417, 'weight_decay': 1.4122812944257067e-05, 'batch_size': 192, 'n_hidden_1': 128, 'dropout_rate_1': 0.16420066820839307}
Best epoch: 40


#### Train Final Model (Depth=1)
Using the best hyperparameters and the optimal number of training epochs found by Optuna, this cell trains the final 2-layer MLP on the entire training dataset. The model's weights (`.pth` file) are saved to the results directory.

In [5]:
mlp_depth1.train_final_model(patience=10) # Stop if loss doesn't improve for 10 epochs

Training mlp_depth1 with depth 1 for 40 epochs with patience of 10 epochs.
Using batch size: 192 and best params: {'learning_rate': 0.01040797308412417, 'weight_decay': 1.4122812944257067e-05, 'batch_size': 192, 'n_hidden_1': 128, 'dropout_rate_1': 0.16420066820839307}.
Epoch    0, Loss: 1.039254
Epoch    1, Loss: 1.000370
Epoch    2, Loss: 0.998688
Epoch    3, Loss: 0.998679
Epoch    4, Loss: 0.994686
Epoch    5, Loss: 0.993515
Epoch    6, Loss: 0.994243
Epoch    7, Loss: 0.996671
Epoch    8, Loss: 0.994295
Epoch    9, Loss: 0.994711
Epoch   10, Loss: 0.994124
Epoch   11, Loss: 0.993570
Epoch   12, Loss: 0.994356
Epoch   13, Loss: 0.994356
Epoch   14, Loss: 0.994505
Epoch   15, Loss: 0.993507
Epoch   16, Loss: 0.993540
Epoch   17, Loss: 0.991932
Epoch   18, Loss: 0.993162
Epoch   19, Loss: 0.992082
Epoch   20, Loss: 0.993136
Epoch   21, Loss: 0.993292
Epoch   22, Loss: 0.994159
Epoch   23, Loss: 0.993268
Epoch   24, Loss: 0.995443
Epoch   25, Loss: 0.994729
Epoch   26, Loss: 0.994273


#### Generate Final Predictions (Depth=1)
Finally, we load the trained 2-layer model and use it to make predictions on the held-out 2020 test set.

In [6]:
# Make predictions on the 2020 test data
preds_depth1 = mlp_depth1.make_final_predictions()
print("\nMLP (Depth=1) Predictions (first 5 rows):")
print(preds_depth1[:5])

mlp_depth1 predictions saved to: /Users/arvindsuresh/Documents/Github/Election-prediction-May-2025/2020-results-20251023/preds/mlp_depth1_preds.csv.

MLP (Depth=1) Predictions (first 5 rows):
[[0.154508   0.29219887 0.02766594 0.5256272 ]
 [0.14840181 0.29685402 0.03713985 0.5176043 ]
 [0.19731696 0.16657394 0.00948197 0.62662715]
 [0.12144238 0.2577286  0.01204269 0.60878634]
 [0.09964711 0.3096592  0.01723056 0.57346314]]
